### Project - Building a Spam Filter (Naive Bayes)

In this project we're going to use the multionomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans to build a spam filter for SMS messages. The outline of our work consists in:
* teach the computer how humans classify messages
* the computer uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam
* finally the computer classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).<br>

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

![Image](https://images.unsplash.com/photo-1557200134-90327ee9fafa?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1740&q=80)
_Photo by Stephen Phillips - Hostreviews.co.uk on Unsplash_

### Exploring the Dataset

In [1]:
import pandas as pd

In [2]:
sms_spam=pd.read_csv("C:/Users/Denisa/Desktop/Project Apps/project 15/SMSSpamCollection",sep='\t',header=None, names=['Label', 'SMS'])
sms_spam

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
sms_spam.shape

(5572, 2)

Finding what percentage of the messages is spam and what percentage is ham ("ham" means non-spam).

In [4]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

### Training and Test Set

To test the spam filter, we're first going to split our dataset into two categories:
* A training set, which we'll use to "train" the computer how to classify messages (about 80% of the dataset)
* A test set, which we'll use to test how good the spam filter is with classifying new messages (about 20% of the dataset)<br>

We are going to design the test set before creating the spam filter to make sure we don't come up with a biased test. We are going to use the messages from the test set as new and have the spam filter classify them to see how accurate the spam filter is.Our goal is to have a spam filter with an accuracy greater than 80%.

In [5]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [6]:
training_set['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test_set['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The percentages of spam and ham in both the training and the test set are similar to what we have in the full dataset.

### Data Cleaning

### Letter Case and Punctuation

In [9]:
#Remove all the punctuation from the SMS column
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ',regex=True)
#For each message, transform every letter in every word to lower case
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

We will create a vocabulary for the messages in the training set. We begin by transforming each message from the SMS column into a list by splitting the string at the space character. We will initiate an empty list named vocabulary, iterate over the SMS column and using a nested loop iterate each message in the SMS column and append each string (word) to the vocabulary list if it doesn't exist already

In [12]:
training_set['SMS'] = training_set['SMS'].str.split()
vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        if word not in vocabulary:
            vocabulary.append(word)


In [13]:
len(vocabulary)

7783

### The Final Training Set

To create the dictionary we need for our training set (that we will then convert to a DataFrame) we start by initializing a dictionary named word_counts_per_sms, where each key is a unique word from the vocabulary, and each value is a list of the length of training set, where each element in the list is a 0. We loop over training_set['SMS'] using at the same time the enumerate() function to get both the index and the SMS message (index and sms). Using a nested loop, we loop over sms and we incremenent word_counts_per_sms[word][index] by 1.

In [14]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1


In [15]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,yep,by,the,pretty,sculpture,yes,princess,are,you,going,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Concatenate the DataFrame above with the DataFrame containing the training set (this way, we'll also have the Label and the SMS columns).

In [16]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating Constants

The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:
\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}
\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}
where
\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:
* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary 

In [ ]:

# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

### Calculating Parameters

Next we want to calculate P(wi|Spam) and P(wi|Ham). We will initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0. Next we will isolate the spam and the ham messages in the training set into two different DataFrames. Finally we will iterate over the vocabulary, and, for each word, calculate P(wi|Spam) and P(wi|Ham)

In [ ]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham
    
ham_messages

### Classifying A New Message

We have all the constants and parameters we need for the spam filter, which can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:<br><br>
1) If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.<br>
2) If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.<br>
3) If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

After erforming a bit of data cleaning on the string message we will iterate over each word in message and for each word, if the word is present in the dictionary containing the spam parameters, then we will update the value of p_spam_given_message by multiplying with the parameter value specific to that word. If the word is present in the dictionary containing 
the ham parameters, then we will update the value of p_ham_given_message by multiplying with the parameter value specific to that word. Finally we will print a classification label.

In [ ]:
import re

def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    #initiate with an initial value
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [ ]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

In [ ]:

classify("Sounds good, Tom, then see u there")

### Measuring the Spam Filter's Accuracy

 We'll now try to determine how well the spam filter does on our test set, given that our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

In [ ]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now we have a function that returns labels instead of printing them, so we can use it to create a new column in our test set.

In [ ]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages.

In [ ]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

The filter had an accuracy of 98.74% on the test set, which is an excellent result.